In [1]:
import numpy as np
import random

from keras.models import load_model

import nltk
from nltk.stem import WordNetLemmatizer

import json

import tkinter as tk
from tkinter import *

In [2]:
model = load_model(filepath = "final_model.h5")

In [3]:
#The "intents" file could also be written directly in the notebook.
with open(file = "Intents.json") as data:
    data = data.read()

intents = json.loads(s = data)

vocabulary = json.load(fp = open(file = "Words.json", mode = "r"))
classes = json.load(fp = open(file = "Classes.json", mode = "r"))

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
def clean_text(input_text):
    '''
    A function that will perform the tokenization and lemmatization processes upon the user's input query.
    
    Parameters:
        input_text (str): The user's input text.
        
    Returns:
        tokens (list): The user's input text after the tokenization and lemmatization processes.
    '''
    
    #Will perform the tokenization upon the user's query.
    tokens = nltk.word_tokenize(text = input_text)
    
    #List comprehension is being used to update the "tokens" list with the lemmatized lowercased version of each word.
    #We do not have to take care of punctuation, as it will be ignored while vectorizing the user's input query.
    tokens = [lemmatizer.lemmatize(word = word.lower()) for word in tokens]
    
    return tokens

In [6]:
def bag_of_words(input_query, vocabulary):
    '''
    A function that will encode the user's input query.
    
    Parameters:
        input_query (str): The user's input text.
        vocabulary (list): The list of all the tokens of each pattern.
        
    Returns:
        bow (numpy array): The encoded version of the user's input text.
    '''
    
    #Will perform the tokenization and lemmatization processes upon the user's query.
    tokens = clean_text(input_text = input_query)
    
    #The NumPy array which will represent the vectorized version of the user's input query.
    bow = np.zeros(shape = len(vocabulary), dtype = "int")
    
    #The encoding will be performed upon user's input query.
    for word in tokens:
        for index, w in enumerate(iterable = vocabulary):
            if word == w:
                bow[index] = 1
                
    return bow

In [7]:
def predict_tag(input_text, vocabulary, model):
    '''
    A function that will produce the tag prediction by the neural network.
    
    Parameters:
        input_text (str): The user's input text.
        vocabulary (list): The list of all the tokens of each pattern.
        model (Sequential): The trained neural network that will perform the prediction process.
        
    Returns:
        str: The predicted tag.
    '''
    
    #Will encode the user's input query.
    bow = bag_of_words(input_query = input_text, vocabulary = vocabulary)
    
    #Will return a sorted NumPy array with the predictions of the neural network about what tag the user’s input query 
    #belongs to.
    result = model.predict(x = np.array(object = [bow]))[0]
    
    #"np.argmax()" will return the index of the highest probability score in the probability array produced by the neural 
    #network.
    return classes[np.argmax(a = result)]

In [8]:
def get_response(predicted_tag, intents_json):
    '''
    A function that will produce the chatbot's random response according to the predicted tag.
    
    Parameters:
        predicted_tag (str): The tag that was predicted by the neural network.
        intents_json (dict): The intents dictionary containing the responses to choose from.
        
    Returns:
        response (str): The random response according to the predicted tag.
    '''
    
    #Will loop through all the intents within the intents JSON file, and will produce random response once the predicted 
    #tag corresponds to the current tag.
    for intent in intents_json["intents"]:
        if intent["tag"] == predicted_tag:
            response = random.choice(seq = intent["responses"])
            break
            
    return response

In [9]:
def chatbot_response(input_text):
    '''
    A function that wraps up all the functionality defined above.
    
    Parameters:
        input_text (str): The user's input text.
        
    Returns:
        chatbot_response (str): The chatbot's random response according to the predicted tag.
    '''
    
    predicted_tag = predict_tag(input_text = input_text, vocabulary = vocabulary, model = model)
    chatbot_response = get_response(predicted_tag = predicted_tag, intents_json = intents)
    
    return chatbot_response

In [18]:
main_window = tk.Tk()
main_window.geometry(newGeometry = "400x500")
main_window.title(string = "Kulo Chatbot Project")
main_window.iconbitmap(bitmap = "KCBPG.ico")
main_window.configure(background = "Light Blue")

#Will create the chat window widget where the user and the chatbot will communicate.
chat_window = Text(master = main_window, background = "White", borderwidth = 1, wrap = "word")

#Specifies the location and the size of the chat window.
chat_window.place(x = 7, y = 7, height = 386, width = 370)

#Will disable the user from writing directly inside the chat window.
chat_window.configure(state = DISABLED)

#Will create the scrollbar widget.
scroll_bar = Scrollbar(master = main_window, command = chat_window.yview)
chat_window["yscrollcommand"] = scroll_bar.set
scroll_bar.place(x = 376, y = 7, height = 386)

#Will create the text window widget where the user will write his queries to the chatbot.
text_window = Text(master = main_window, background = "White", borderwidth = 1, font = ("Helvetica", 10))

#Specifies the location and the size of the text window.
text_window.place(x = 128, y = 401, height = 90, width = 265)

def send_text():
    '''
    A function that will generate the chatbot's response according to the user's input text.
    '''
    
    #Will receive the user's input text.
    input_text = text_window.get(index1 = "0.0", index2 = END).capitalize()
    
    #Will delete the user's input text from the text window after the text was sent.
    text_window.delete(index1 = "0.0", index2 = END)
    
    #Will enable the chat window to change his state, in the sense that the user’s input text will appear within the chat
    #window.
    chat_window.configure(state = NORMAL)
        
    #Will insert the user’s input text at the end of the conversation so far.
    chat_window.insert(index = END, chars = "You: " + input_text + "\n")
    
    #Specifies the appearance of the text within the chat window.
    chat_window.configure(font = ("Helvetica", 11, "bold"), foreground = "#1E90FF")
    
    if len(input_text) > 2:
        response = chatbot_response(input_text = input_text)
        
    else:
        response = random.choice(seq = intents["intents"][1]["responses"])
        
    #Will insert the chatbot’s response at the end of the conversation so far.
    chat_window.insert(index = END, chars = "Kulo: " + response + "\n\n")
    
    #Will disable any changes to occur within the chat window until the next message will be sent.
    chat_window.config(state = DISABLED)

#Will create a PhotoImage object, so later we can display the image on the Button widget.
send_icon = tk.PhotoImage(file = "SI.png")

#Will create the send button with the functionality of the “send_text()” function discussed above.
send_button = Button(master = main_window, command = send_text, image = send_icon, 
                    background = "#2F4F4F", borderwidth = 2, font = ("Helvetica", 12, "bold"), width = 50)

#Specifies the location and the size of the send button.
send_button.place(x = 37, y = 420, height = 50)

#Wherever there is a use of the "place" method, it can be replaced with the "pack" or "grid" methods as well.

main_window.mainloop()